# Предсказание энергии и сил с помощью нейронного потенциала

In [1]:
from ase.io import read, write
from ase.visualize import view

## MACE

In [12]:
import mace_models

In [16]:
model = mace_models.load("MACE-MP-0_small")
calc = model.get_calculator()


        You're using the MACE-MP-0_small model. The model is released under the MIT license.
        Note:
        If you are using this model, please cite the relevant paper for the Materials Project,
        any paper associated with the MACE model, and also the following:
        - MACE-Universal by Yuan Chiang, 2023, Hugging Face, Revision e5ebd9b,
            DOI: 10.57967/hf/1202, URL: https://huggingface.co/cyrusyc/mace-universal
        - Matbench Discovery by Janosh Riebesell, Rhys EA Goodall, Philipp Benner, Yuan Chiang,
            Alpha A Lee, Anubhav Jain, Kristin A Persson, 2023, arXiv:2308.14920
        - https://arxiv.org/abs/2401.00096
           
Default dtype float32 does not match model dtype float64, converting models to float32.


In [17]:
# --- 1. Read xyz with ASE ---
mol = read("./data/Adamantane.sdf")   # ASE Atoms object
mol.calc = calc
# --- 3. Compute properties ---
energy = mol.get_potential_energy()      # in eV
forces = mol.get_forces()                # shape (n_atoms, 3)

print("Energy [eV]:", energy)
print("Forces [eV/Å]:", forces)


Energy [eV]: -150.65699768066406
Forces [eV/Å]: [[-0.12354073 -0.14099595 -0.296571  ]
 [ 0.13383727  0.31513548 -0.06785819]
 [ 0.24869493 -0.20740423  0.12984572]
 [-0.2595753   0.03201146  0.23448575]
 [ 0.00460589  0.07350921 -0.1508133 ]
 [ 0.04659631 -0.144941   -0.06730784]
 [-0.15940371 -0.04533143 -0.02577613]
 [ 0.15655166  0.04350179  0.02453544]
 [-0.05006527  0.14558153  0.06975335]
 [-0.00345007 -0.07374209  0.1469714 ]
 [-0.04331528 -0.04862058 -0.102209  ]
 [ 0.04789099  0.11189583 -0.02425441]
 [ 0.08854449 -0.07382677  0.04648544]
 [-0.09074399  0.01122495  0.08151346]
 [ 0.08243969 -0.00980102 -0.07544948]
 [-0.07981873  0.0666938  -0.04269473]
 [-0.0416047  -0.1003322   0.02060886]
 [ 0.08406468 -0.01139742 -0.0758651 ]
 [-0.08088805  0.06568318 -0.0418711 ]
 [-0.04419085 -0.10051446  0.02123776]
 [ 0.08512318 -0.00972746 -0.07481712]
 [ 0.04235387  0.04639164  0.09635566]
 [ 0.03887702  0.04459685  0.09303236]
 [-0.08022614  0.06753297 -0.04122199]
 [-0.04367642 -0

## Nabla2DFT

In [1]:
import os

import hydra
import pytorch_lightning as pl
import torch
from ase.db import connect
from nablaDFT import model_registry
from nablaDFT.dataset import PyGNablaDFTDataModule
from nablaDFT.pipelines import predict
from omegaconf import OmegaConf

In [2]:
data_args = {
    "root": "./datasets/nablaDFT/test",
    "dataset_name": "dataset_test_conformations_tiny",
    "batch_size": 4,
    "num_workers": 2,
}


In [3]:
model_registry.list_models()

['SchNet_train_large',
 'SchNet_train_medium',
 'SchNet_train_small',
 'SchNet_train_tiny',
 'PaiNN_train_large_traj_medium',
 'PaiNN_train_large',
 'PaiNN_train_medium',
 'PaiNN_train_small',
 'PaiNN_train_tiny',
 'DimeNet++_train_large',
 'DimeNet++_train_medium',
 'DimeNet++_train_small',
 'DimeNet++_train_tiny',
 'PhiSNet_train_large',
 'PhiSNet_train_medium',
 'PhiSNet_train_small',
 'PhiSNet_train_tiny',
 'SchNOrb_train_large',
 'SchNOrb_train_medium',
 'SchNOrb_train_small',
 'SchNOrb_train_tiny',
 'GemNet-OC_train_large',
 'GemNet-OC_train_medium',
 'GemNet-OC_train_small',
 'GemNet-OC_train_tiny',
 'GemNet-OC_train_large_traj_medium',
 'ESCN-OC_train_large',
 'ESCN-OC_train_medium',
 'ESCN-OC_train_small',
 'ESCN-OC_train_tiny',
 'Equiformer-v2_train_large',
 'Equiformer-v2_train_medium',
 'Equiformer-v2_train_small',
 'Equiformer-v2_train_tiny',
 'Graphormer3D-small_train_large',
 'Graphormer3D-small_train_medium',
 'Graphormer3D-small_train_small',
 'Graphormer3D-small_train

In [7]:
model = model_registry.get_pretrained_model("lightning", "DimeNet++_train_large")
datamodule = PyGNablaDFTDataModule(**data_args)
trainer = pl.Trainer(accelerator=accelerator, devices=devices)
trainer.logger = False


KeyError: 'DimeNet++'

In [8]:
model_registry.get_pretrained_model?

Signature: model_registry.get_pretrained_model(model_type: str, model_name: str)
Docstring:
Instantiates model and restores model's state from checkpoint.

Downloads model checkpoint if necessary.
.. note::
    Not supported for SchNorb and PhiSNet models.

Args:
    model_type (str): model framework, must be one of ["torch", "lightning"]
    model_name (str): model checkpoint name. Available models can be listed with
        :meth:nablaDFT.registry.ModelRegistry.list_models
File:      ~/anaconda3/envs/p4env/lib/python3.10/site-packages/nablaDFT/model_registry.py
Type:      method